In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
#from astropy.modeling import models, fitting
#from astropy.stats import sigma_clip
#import scipy.signal as sgn
#import pandas as pd
#df=pd.read_csv('Part II/-5_B7.5/F0000CH2.csv', sep=',', header = None)
#print(df)

In [ ]:
#reading out the csv and returning time and voltage as arrays
def grabData(file):
    #file is string with form :
    #'Part II/-<absolute detuning value>_B<value of the current>/F<# of csv with 4 digits>CH1.csv'
    #for example 'Part II/-6_B7.5/F0001.csv'
    dg = np.genfromtxt(file, delimiter=',')
    #6 dim array with most of them being 'empty' caused by the way Oscilloscope writes the .csv
    #Example of the first 3 'lines', column 0 and 1 become NaN as well after 17 lines
#                   0                   1   2      3     4   5
#0        Record Length        2.500000e+03 NaN  0.000  3.04 NaN
#1      Sample Interval        1.000000e-03 NaN  0.001  3.02 NaN
#2        Trigger Point  0.000000000000e+00 NaN  0.002  3.02 NaN
    g0,g1,g2,g3,g4,g5 = np.hsplit(dg,6)
    #g3=time, g4=voltage
    return g3,g4
#fit function for background noise
def con(x,b):
    return 0*x+b
def conversion():
    

In [ ]:
#amount of csv files
n=32 
#array for loop
narr=np.arange(n+1,dtype=int) 
#Numbering for the files with given current
current75=([0,4,8,12,16,20,24,29]) 
current8=([1,5,9,13,17,21,25,30])
current85=([2,6,10,14,18,22,27,31])
current9=([3,7,11,15,19,23,28,32])

In [ ]:
#Determinging the Background caused by the Laserlight also being measured by the ccd
xlin=np.arange(4)
t0b, background = grabData('Part II/Background/F0033CH1.csv') #change folder 'Part II if needed'
t1b = t0b[np.where(background > 0.025)]
back = background[np.where(background > 0.025)]
xbb = t0b[np.where(background > 0.043)]
ybb = background[np.where(background > 0.043)]
ubb, fbb = curve_fit(con,xbb,ybb)
xb = t0b[np.where(background < 0.043)]
yb = background[np.where(background < 0.043)]
xbd = xb[np.where(yb > 0.025)]
ybd = yb[np.where(yb > 0.025)]
ubd, fbd = curve_fit(con,xbd,ybd)
print(np.size(t0),np.size(xb),np.size(xbd))
print(ubd)
plt.scatter(t1b,back,marker="|")
plt.plot(xlin,con(xlin,ubd),'red',linestyle='--')
plt.plot(xlin,con(xlin,ubb),'red',linestyle='--')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.xlim(0,2.5)
plt.ylim(0.02,0.06)
plt.title('Background from just Laserlight')
plt.gcf().set_size_inches(11.69, 8.27)
plt.show()

In [ ]:
#Reading out all 33 Files
j=1
t_all = np.empty((n,2500,1))
u_all = np.empty((n,2500,1))
for i in narr: #need to change all different strings to match ur folders
    if(i==26): #removed cause we have this data set 2 times and this one was worse
        continue
    if(i<10): 
        c='F000'+str(i)
    else:
        c='F00'+str(i)
    if(i in current75):
        b='7.5'
        r=1
    if(i in current8):
        b='8'
        r=2
    if(i in current85):
        b='8.5'
        r=3
    if(i in current9):
        b='9'
        r=4
    if(i<4):
        a=str(5)
        z=1
    else: 
        if(i<8):
            a=str(6)
            z=2
        else:
            if(i<12):
                a=str(7)
                z=3
            else:
                if(i<16):
                    a=str(8)
                    z=4
                else:
                    if(i<20):
                        a=str(10)
                        z=5
                    else:
                        if(i<24):
                            a=str(12)
                            z=6
                        else:
                            if(i<29):
                                a=str(14)
                                z=7
                            else:
                                a=str(16)
                                z=8
    path = 'Part II/-'+a+'_B'+b+'/'+c+'CH1.csv' 
    t,u = grabData(path) #everything up till here was just to read out the .csv
    if(i<26):
        u_all[i]=u
        t_all[i]=t

    else:
        u_all[i-1]=u      
        t_all[i-1]=t

    t = t[np.where(u > 0.025)]
    u = u[np.where(u > 0.025)]
    #u=u-background
    
    plt.subplot(2,4,j)
    ymax=np.amax(u)+0.03
    plt.scatter(t,u,marker="|") 
    plt.xlim(t[0],t[-1])
    plt.ylim(0.01,ymax)
    plt.xlabel('Time')
    plt.ylabel('Voltage')
    plt.title('I='+b+', $\delta$='+a)
    j=j+1
    if(j>8): 
        plt.subplots_adjust(wspace=0.4,hspace=0.8)
        plt.gcf().set_size_inches(11.69, 8.27/1.3)
        plt.show()
        j=1
    del t,u,z,r,a,b,c,path